In [ ]:
import  numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings('ignore')


## Steps 1 --->

df.duplicated().sum()

### Step 2  EDA ---->



### Step 3
### Outlier Handling ------>

def detect_outliers_in_columns(df):

    outlier_columns = []
    numerical_cols = df.select_dtypes(include=['int64']).columns

    for col in numerical_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR


        if ((df[col] < lower_bound) | (df[col] > upper_bound)).any():
            outlier_columns.append(col)

    return outlier_columns



def count_of_outliers(df, outlier_columns): 
    outlier_counts = {}
    for col in outlier_columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outlier_count = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()
        outlier_counts[col] = outlier_count
    return outlier_counts


def cap_outliers(df, outlier_columns):
    for col in outlier_columns:
        if col != 'Response':
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
    

            df.loc[df[col] < lower_bound, col] = lower_bound
            df.loc[df[col] > upper_bound, col] = upper_bound
    
    return df


outlier_columns = detect_outliers_in_columns(df)
print("Columns with outliers:", outlier_columns)
outlier_counts = count_of_outliers(df, outlier_columns)
print("Outlier counts per column:", outlier_counts)
df_capped = cap_outliers(df, outlier_columns)
outlier_counts = count_of_outliers(df, outlier_columns)
print("Outlier counts per column:", outlier_counts)

#############################################################################################

df = df_capped.copy()
del df_capped


#############################################################################################

# train['Job Satisfaction'] = train['Job Satisfaction'].fillna(train['Job Satisfaction'].median())
# train.isnull().sum()/train.shape[0]*100
# train['Job Satisfaction'] = train['Job Satisfaction'].fillna(train['Job Satisfaction'].mode()[0])



# For Numerical
# # skew

# -0.5 to 0.5  -- mean
# -1 and -0.5 or 0.5 and 1 --> median
# less than -1 and greater than 1  --> median

# For categorical
## Nominal data --> mode
## Ordinal data --> median

#############################################################################################

### Encoding
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder

lab = LabelEncoder()
df['Gender' ]  = lab.fit_transform(df['Gender'])
df['Vehicle_Damage' ]  = lab.fit_transform(df['Vehicle_Damage'])

vehical_age = ['< 1 Year', '1-2 Year', '> 2 Years']  
ordinal_encoder = OrdinalEncoder(categories=[vehical_age])
df['Vehicle_Age'] = ordinal_encoder.fit_transform(df[['Vehicle_Age']])

df_encoded = pd.get_dummies(df, columns=['Vehicle_Age'], drop_first=True)  
bool = df_encoded.select_dtypes(include=['bool']).columns
df_encoded[bool] = df_encoded[bool].astype(int)

#############################################################################################

## Independent and dependence data

y = df_encoded['Response']
x = df_encoded.drop(columns = ['Response'])

#############################################################################################

from sklearn.preprocessing import  StandardScaler

from sklearn.preprocessing import  StandardScaler
scaler = StandardScaler()
x_train_fit = scaler.fit_transform(x)
x_train_scaled = pd.DataFrame(x_train_fit, columns=x.columns)


#############################################################################################

### Train Test Split

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=9,stratify=y)



#############################################################################################

### Balancing datasets

from imblearn.over_sampling import SMOTE

# Applying SMOTE for oversampling
smote = SMOTE(random_state=42)
x_train, y_train = smote.fit_resample(x_train, y_train)


######################################################################################
### Model Part --> Classification

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.pipeline import Pipeline


### Classification
models = [
    ('Logistic Regression', LogisticRegression()),

    # ('Support Vector Machine', SVC()),

    # ('Decision Tree', DecisionTreeClassifier()),
    # ('Random Forest', RandomForestClassifier()),


    ('Gradient Boosting', GradientBoostingClassifier()),
    ('XGBoost', XGBClassifier()),
    ('LightGBM', LGBMClassifier()),
    ('CatBoost', CatBoostClassifier()),

]



from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc , accuracy_score


######################################################################################
## CLassification


for name, model in models:
    pipeline = Pipeline(steps=[('classifier', model)])
    pipeline.fit(x_train_scaled, y_train)
    if hasattr(model, "predict_proba"):
        y_train_pred_prob = pipeline.predict_proba(x_train_scaled)[:, 1]
        fpr_train, tpr_train, thresholds_train = roc_curve(y_train, y_train_pred_prob)
        roc_auc_train = auc(fpr_train, tpr_train)
        plt.figure(figsize=(8, 6))
        plt.plot(fpr_train, tpr_train, color='blue', lw=2, label=f'Train ROC curve (AUC = {roc_auc_train:.2f})')
        plt.plot([0, 1], [0, 1], color='red', linestyle='--')  # Diagonal line
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.0])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'{name} Train ROC Curve')
        plt.legend(loc='lower right')
        plt.grid()
        plt.show()

        print(f'{name} Train AUC: {roc_auc_train:.2f}')
        y_test_pred_prob = pipeline.predict_proba(x_test_scaled)[:, 1]
        fpr_test, tpr_test, thresholds_test = roc_curve(y_test, y_test_pred_prob)
        roc_auc_test = auc(fpr_test, tpr_test)


        plt.figure(figsize=(8, 6))
        plt.plot(fpr_test, tpr_test, color='green', lw=2, label=f'Test ROC curve (AUC = {roc_auc_test:.2f})')
        plt.plot([0, 1], [0, 1], color='red', linestyle='--')  # Diagonal line
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.0])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'{name} Test ROC Curve')
        plt.legend(loc='lower right')
        plt.grid()
        plt.show()


        print(f'{name} Test AUC: {roc_auc_test:.2f}')

    else:
        print(f'{name} does not support predict_proba. Skipping ROC curve plotting.')
        y_train_pred = pipeline.predict(x_train_scaled)
        print(f'{name} Train Accuracy: {accuracy_score(y_train, y_train_pred):.2f}')

        y_test_pred = pipeline.predict(x_test_scaled)
        print(f'{name} Test Accuracy: {accuracy_score(y_test, y_test_pred):.2f}')



######################################################################################



###################################################################################################################
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,ExtraTreesClassifier,AdaBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.feature_selection import RFECV # Recursive Feature Elimination with Cross-Validation.
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier,Pool
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from imblearn.over_sampling import SMOTE
from category_encoders import CatBoostEncoder
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,roc_curve,recall_score,precision_score,f1_score,balanced_accuracy_score,roc_auc_score,ConfusionMatrixDisplay



## Optuna code

# Define the objective function for Optuna
def objective(trial):
    # Choose which model to optimize
    model_name = trial.suggest_categorical("model", ["GradientBoosting", "XGBoost", "LightGBM", "CatBoost"])

    if model_name == "GradientBoosting":
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 50, 500),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.3),
            "subsample": trial.suggest_uniform("subsample", 0.5, 1.0)
        }
        model = GradientBoostingRegressor(**params)

    elif model_name == "XGBoost":
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 50, 500),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.3),
            "subsample": trial.suggest_uniform("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.5, 1.0)
        }
        model = XGBRegressor(**params, random_state=42)

    elif model_name == "LightGBM":
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 50, 500),
            "max_depth": trial.suggest_int("max_depth", -1, 10),
            "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.3),
            "num_leaves": trial.suggest_int("num_leaves", 20, 150),
            "subsample": trial.suggest_uniform("subsample", 0.5, 1.0)
        }
        model = LGBMRegressor(**params, random_state=42)

    elif model_name == "CatBoost":
        params = {
            "iterations": trial.suggest_int("iterations", 100, 500),
            "depth": trial.suggest_int("depth", 3, 10),
            "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.3),
        }
        model = CatBoostRegressor(**params, random_state=42, verbose=0)

    # Train the model
    model.fit(x_train_scaled, y_train)

    # Predict on the test set
    y_pred = model.predict(x_test_scaled)

    # Evaluate using Mean Squared Error
    mse = mean_squared_error(y_test, y_pred)
    return mse

# Create the Optuna study and optimize
study = optuna.create_study(direction="minimize", sampler=TPESampler())
study.optimize(objective, n_trials=50)

# Print the best trial
print("Best trial:")
print(f"MSE: {study.best_value:.4f}")
print("Best hyperparameters:", study.best_trial.params)


######################################################################################
### Feature Importance

best_model = study.best_trial.user_attrs['best_model']

feature_importance = best_model.get_feature_importance()

plt.figure(figsize=(10, 6))
plt.barh(range(len(feature_importance)), feature_importance, align="center")
plt.yticks(range(len(feature_importance)), [f"Feature {i}" for i in range(len(feature_importance))])
plt.xlabel('Feature Importance')
plt.ylabel('Features')
plt.title('CatBoost Feature Importance')
plt.show()


#######################

cm = confusion_matrix(y_test,y_pred)


################################################


from sklearn.decomposition import PCA


pca = PCA(n_components=0.95)  
X_pca = pca.fit_transform(X_scaled)
print(f"Explained variance ratio: {pca.explained_variance_ratio_}")
print(f"Total variance explained by the selected components: {np.sum(pca.explained_variance_ratio_)}")
if X_pca.shape[1] == 2:
    plt.figure(figsize=(8, 6))
    plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='viridis')
    plt.colorbar()
    plt.xlabel('First Principal Component')
    plt.ylabel('Second Principal Component')
    plt.title('PCA of Iris Dataset (95% Variance Retained)')
    plt.show()
else:
    print(f"Number of components retained: {X_pca.shape[1]}")


#####################################################################
### Cross Validation

from sklearn.model_selection import cross_val_score
model = LogisticRegression(max_iter=200)
cv_scores = cross_val_score(model, X_scaled, y, cv=5)

#####################################################################
### Feture importance

# Get feature importances
importances = model.feature_importances_

# Display the feature importances
features = data.feature_names
importance_df = pd.DataFrame({'Feature': features, 'Importance': importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)
print(importance_df)



##################################################################################################################################################


from sklearn.feature_selection import mutual_info_classif

# Assuming X is your features and y is the target
mi = mutual_info_classif(X, y)

# Display the mutual information scores
importance_df = pd.DataFrame({'Feature': X.columns, 'Mutual Information': mi})
importance_df = importance_df.sort_values(by='Mutual Information', ascending=False)
print(importance_df)



from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# Assuming X and y are your features and target variables
model = LogisticRegression(max_iter=200)
rfe = RFE(model, n_features_to_select=1)  # Rank all features
rfe.fit(X, y)

# Get ranking of features
ranking = rfe.ranking_

# Display the ranked features
importance_df = pd.DataFrame({'Feature': X.columns, 'Ranking': ranking})
importance_df = importance_df.sort_values(by='Ranking', ascending=True)
print(importance_df)
